<a href="https://colab.research.google.com/github/UiinKim/Music_Generation/blob/main/audio_transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install audioldm

In [ ]:
import torch
from audioldm.latent_diffusion.ema import *
from audioldm.variational_autoencoder.modules import Encoder, Decoder
from audioldm.variational_autoencoder.distributions import DiagonalGaussianDistribution

from audioldm.hifigan.utilities import get_vocoder, vocoder_infer

class AutocoderKL(nn.Module):
    def __init__(
        self,
        ddconfig=None, #인코더, 디코더의 설정 정보
        lossconfig=None, #손실함수 설정 정보
        image_key='fbank', #입력 데이터의 유형 설정. fbank(필터뱅크),stft(단시간 푸리에 변환) --> 바꿔보기
        embed_dim=None, #latent space의 차원
        time_shuffle=1,
        subband=1, #subband 분해를 제어, 다양한 주파수 대역을 분리하여 처리 가능
        ckpt_path=None, #모델 가중치를 로드하기 위한 체크포인트 파일 경로
        reload_from_ckpt=None,
        ignore_keys=[],
        colorize_nlabels=None,
        monitor=None,
        base_learning_rate=1e-5,
        scale_factor=1
  ):
        super().__init__()

        self.encoder=Encoder(**ddconfig)
        self.decoder=Decoder(**ddconfig)

        self.subband=int(subband)

        if self.subband>1:
          print("Use subband decomposition %s" % self.subband)

        #양자화 컨볼루션 : latent vecotr로 변환하기 위한 컨볼루션 레이어
        self.quant_conv=torch.nn.Conv2d(2*ddconfig['z_channels'],2*embed_dim,1)
        self.post_quant_conv=torch.nn.Conv2d(embed_dim, ddconfig["z_channels"], 1)

        self.vocoder=get_vocoder(None, 'cpu') #HiFi-GAN을 사용하여 스펙트로그램과 같은 오디오 표현을 파형으로 변환하는 역할인 vocoder을 초기화
        self.embed_dim=embed_dim

        if monitor is not None: #모니터링 옵션 설정
          self.monitor=monitor

        self.time_shuffle=time_shuffle
        self.reload_from_ckpt=reload_from_ckpt
        self.reloaded=False
        self.mean, self.std=None, None

        self.scale_factor=scale_factor

    def encode(self, x):
      #x=self.time_shuffle_operation(x)
      x=self.freq_split_subband(x) #서브밴드 분해가 활성화일 경우, input audio feature을 개별 주파수 밴드로 나눈다
      h=self.encoder(x) #인코더로 input data 처리
      moments=self.quant_conv(h) #양자화 컨볼루션을 통해 가우시안 분포의 모멘트(평균과 로그 분산) 생성
      posterior=DiagonalGaussianDistribution(moments) #모멘트로부터 latent vector샘ㅍ플링
      return posterior



